In [42]:
import numpy as np 
import tensorflow as tf
import pdb
import math 

In [2]:
def build_code_table(bits=8, lo=-1.5, hi=1.5):
    code_dict = {}
    half_bins = int(2**bits/2)
    prev_code_array = np.zeros([1,bits],dtype=np.dtype(int)).squeeze()
    prev_code = ''.join(map(str, prev_code_array))
    step = hi/half_bins
    v_range = [0.0, step]
    code_dict[prev_code] = v_range
    for i in range(half_bins-1): 
        print('i:',i)
        idx = bits-1
        done = False
        right_code = str()
        while not done:
            targ_bit = int(prev_code[idx])
            targ_bit += 1
            if targ_bit == 2:
                right_code = str(0) + right_code 
                idx -= 1
                done = False
            else: 
                right_code = str(targ_bit) + right_code 
                done = True
        code = prev_code[:idx] + right_code 
        v_range = [v_range[1], v_range[1]+step]
        code_dict[code] = v_range
        prev_code = code[:]
    
    
    prev_code_array = np.zeros([1,bits],dtype=np.dtype(int)).squeeze()
    prev_code_array[0] = 1
    prev_code = ''.join(map(str, prev_code_array))
    step = lo/half_bins
    v_range = [0.0, step]
    code_dict[prev_code] = v_range
    for i in range(half_bins-1): 
        print('i:',i)
        idx = bits-1
        done = False
        right_code = str()
        while not done:
            targ_bit = int(prev_code[idx])
            targ_bit += 1
            if targ_bit == 2:
                right_code = str(0) + right_code 
                idx -= 1
                done = False
            else: 
                right_code = str(targ_bit) + right_code 
                done = True
        code = prev_code[:idx] + right_code 
        v_range = [v_range[1], v_range[1]+step]
        code_dict[code] = v_range
        prev_code = code[:] 
        print(prev_code)
    return code_dict 

In [4]:
def real2bits(value,bits=8,radius=1.5):
    binary = np.zeros([bits,])
    step = radius/(2**bits/2)
    if value < 0:
        binary[0] = 1
    integer = int(np.floor(np.abs(value/step)))
    done = False 
    i = bits-1
    while not done: 
        binary[i] = integer % 2
        integer = int(integer/2)
#         pdb.set_trace()
        if integer == 0:
            done = True 
        else:
            i -= 1 
    return binary

In [4]:
step = 1.5/2**(8-1)
N = -1
inp = N*step
bit_array = real2bits(inp,bits=8,radius=1.5)

In [5]:
def bits2real(bit_array,bits=8,radius=1.5):
    step = radius/(2**(bits-1))
    weights = np.zeros([bits,])
#     weights[0] = -1.0
    for i in range(bits-1):
        weights[bits-i-1] = 2**i
    value = weights.dot(bit_array)*step
    if bool(bit_array[0]):
        value *= -1.0
    return value

In [6]:
def run_model(session, output, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(output, y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [loss_val,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now }
            # get batch size
            actual_batch_size = yd[i:i+batch_size].shape[0]
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            pdb.set_trace()
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct

In [7]:
def basic_MLP(X,y,sizes,is_training,bits=8,threshold=0,dropout=False): 
    layers = {}
    layers[0] = tf.layers.dense(X, sizes[0],
                                activation = tf.nn.relu,
                                use_bias=True,
                                )
    if dropout:
        dropout_key = str(0)+'d' 
        layers[dropout_key] = tf.layers.dropout(layers[0],
                                                rate=0.25,
                                                training=is_training)
    for j,  size in enumerate(sizes[1:]): 
        i = j+1
        if dropout: 
            dropout_key_prior = str(i-1)+'d' 
            layers[i] = tf.layers.dense(layers[dropout_key_prior], size,
                                    activation = tf.nn.relu,
                                    use_bias=True,
                                    )
            dropout_key = str(i)+'d' 
            layers[dropout_key] = tf.layers.dropout(layers[dropout_key],
                                                rate=0.25,
                                                training=is_training)
        else:
            layers[i] = tf.layers.dense(layers[i-1], size,
                                    activation = tf.nn.relu,
                                    use_bias=True,
                                    )

    output_value = tf.layers.dense(layers[len(sizes)-1], bits,
                                   activation = tf.nn.relu,
                                   use_bias=False,
                                    ) #used relu to guarentee no negative values
    
    output = tf.round(output_value)
    
#     threshold_list = [threshold]*bits
#     threshold_tensor = tf.constant(threshold_list)
#     output_bool = tf.greater_equal(output_value, threshold_tensor)
#     output = tf.cast(output_bool,dtype=tf.uint8) 
    
    return output


In [6]:
def tf_bits2real(X, bits=8,radius=1.5): #FIXME 
    step = radius/(2**(bits-1))
    weight_list = [0]
    for i in range(bits-1):
        power = bits - 2 - i 
        weight_list.append(2**power)
    weight_list = np.array(weight_list)
    weight_list = weight_list.reshape([-1,1])
#     pdb.set_trace()
    conversion_tensor = tf.constant(weight_list,
                                   dtype=tf.float32)
#     pdb.set_trace()
    real_output = tf.matmul(X, conversion_tensor,
                            transpose_a=False)
    idx_array = np.zeros([bits,1])
    idx_array[0] = 1
    idx_tensor = tf.constant(idx_array,
                            dtype=tf.float32)
    neg = tf.matmul(X,idx_tensor)
    neg_factor = tf.multiply(neg, tf.constant(-2, dtype=tf.float32))
    scale = neg_factor + 1.0
    real_output = tf.multiply(real_output, scale)*step
#     tf.cond(tf.cast(neg,tf.bool), )
    return real_output

In [9]:
SIZES = [64, 128, 256]
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[1,1])
y = tf.placeholder(tf.float32, shape=[1,1])
is_training = tf.placeholder(tf.bool, shape=1)
output = basic_MLP(X,y,SIZES,is_training,bits=8,threshold=0,dropout=False)
real_output = tf_bits2real(output,bits=8)
# pdb.set_trace()
difference = real_output - y
loss = tf.sqrt(tf.square(difference))
optimizer = tf.train.AdamOptimizer(1e-3) 
train_step = optimizer.minimize(loss)

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ['Tensor("dense/kernel/read:0", shape=(1, 64), dtype=float32)', 'Tensor("dense/bias/read:0", shape=(64,), dtype=float32)', 'Tensor("dense_1/kernel/read:0", shape=(64, 128), dtype=float32)', 'Tensor("dense_1/bias/read:0", shape=(128,), dtype=float32)', 'Tensor("dense_2/kernel/read:0", shape=(128, 256), dtype=float32)', 'Tensor("dense_2/bias/read:0", shape=(256,), dtype=float32)', 'Tensor("dense_3/kernel/read:0", shape=(256, 8), dtype=float32)'] and loss Tensor("Sqrt:0", shape=(1, 1), dtype=float32).

In [2]:
# https://stackoverflow.com/questions/38833934/write-custom-python-based-gradient-function-for-an-operation-without-c-imple

'1.2.1'

In [6]:
from tensorflow.python.framework import function
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops 

ModuleNotFoundError: No module named 'tensorflow'

In [29]:
# def windowgrad(x,dy,center,width):
#     lo = center - width/2.0
#     hi = center + width/2.0
#     lo = 1.0 - width/2.0
#     hi = 1.0 + width/2.0
# @function.Defun(tf.float32, tf.float32)
# def windowgrad(x,dy):
#     lo = tf.constant(0.5)
#     hi = tf.constant(1.5)
#     lo_mask = tf.greater_equal(x,lo)
#     lo_mask = tf.cast(lo_mask,dtype=tf.float32)
#     hi_mask = tf.less_equal(x,hi)
#     hi_mask = tf.cast(hi_mask,dtype=tf.float32)
#     grad = tf.constant(1.0, dtype=tf.float32,
#                       shape=x.shape)
#     grad = tf.multiply(grad, lo_mask)
#     grad = tf.multiply(grad, hi_mask)
#     grad = tf.multiply(grad,dy)
#     return grad

# @function.Defun(tf.float32, grad_func=windowgrad) # not needed because not a python function
def window(x,center,width):
    lo = center - width/2.0
    hi = center + width/2.0
#     lo = tf.constant(0.5)
#     hi = tf.constant(1.5)
    lo_mask = tf.greater_equal(x,lo)
    lo_mask = tf.cast(lo_mask,dtype=tf.float32)
    hi_mask = tf.less_equal(x,hi)
    hi_mask = tf.cast(hi_mask,dtype=tf.float32)
    out = tf.multiply(x, lo_mask)
    out = tf.multiply(out, hi_mask)
    return out 

# @ops.RegisterGradient("window")
# def _windowgrad(op, grad):
# # def windowgrad(x,dy,center,width):
# #     lo = center - width/2.0
# #     hi = center + width/2.0
# #     lo = 1.0 - width/2.0
# #     hi = 1.0 + width/2.0
#     x = op.inputs[0]
#     lo = tf.constant(0.5)
#     hi = tf.constant(1.5)
#     lo_mask = tf.greater_equal(x,lo)
#     hi_mask = tf.less_equal(x,hi)
#     win_grad = tf.constant(1.0, dtype=tf.float32,
#                       shape=x.shape)
#     win_grad = tf.boolean_mask(win_grad, lo_mask)
#     win_grad = tf.boolean_mask(win_grad, hi_mask)
#     win_grad = tf.multiply(win_grad, grad)
#     return [win_grad]

In [43]:
def basic_MLP_wcustom(X,y,sizes,is_training,bits=8,threshold=0,dropout=False): 
    layers = {}
    layers[0] = tf.layers.dense(X, sizes[0],
                                activation = tf.nn.relu,
                                use_bias=True,
                                )
    if dropout:
        dropout_key = str(0)+'d' 
        layers[dropout_key] = tf.layers.dropout(layers[0],
                                                rate=0.25,
                                                training=is_training)
    for j,  size in enumerate(sizes[1:]): 
        i = j+1
        if dropout: 
            dropout_key_prior = str(i-1)+'d' 
            layers[i] = tf.layers.dense(layers[dropout_key_prior], size,
                                    activation = tf.nn.relu,
                                    use_bias=True,
                                    )
            dropout_key = str(i)+'d' 
            layers[dropout_key] = tf.layers.dropout(layers[dropout_key],
                                                rate=0.25,
                                                training=is_training)
        else:
            layers[i] = tf.layers.dense(layers[i-1], size,
                                    activation = tf.nn.tanh,
                                    use_bias=True,
                                    )

    output_value = tf.layers.dense(layers[len(sizes)-1], bits,
                                   activation = None,
                                   use_bias=False,
                                    ) #used relu to guarentee no negative values
#     pdb.set_trace()
    center = tf.ones([1],dtype=tf.float32)
    width = tf.constant(1.0,dtype=tf.float32)
#     output = window(output_value,center,width)
    output = output_value
    return output


In [49]:
SIZES = [64, 128, 256]
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None,1],name='Input')
y = tf.placeholder(tf.float32, shape=[None],name='Correct_output')
is_training = tf.placeholder(tf.bool,name='training')
output = basic_MLP_wcustom(X,y,SIZES,is_training,bits=8,threshold=0,dropout=False)
real_output = tf_bits2real(output,bits=8)
# pdb.set_trace()
difference = real_output - y
loss = tf.square(difference)
mean_loss = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(1e-6) 
train_step = optimizer.minimize(mean_loss)

In [50]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

xd = np.arange(0.0, 1.49, 0.01)
# np.random.shuffle(xd)
yd = np.zeros_like(xd)
for i, x in enumerate(xd):
    yd_ = real2bits(x, bits=8, radius=1.5)
    yd[i] = bits2real(yd_, bits=8, radius=1.5)
#     pdb.set_trace()
xd = xd.reshape([-1,1])

In [60]:
for i in range(20000):
    idx = np.random.randint(0,xd.shape[0]-1,100)
    ml, _ = sess.run([mean_loss, train_step],feed_dict={X: xd[idx,:], y: yd[idx]})
    if i % 1000 == 0:
        print(ml)

# total_loss, total_correct = run_model(sess, real_output, loss, xd, yd,
#                                       epochs=1, batch_size=64, print_every=100,
#                                       training=train_step, plot_losses=True)

# iter_cnt,loss,np.sum(corr)/actual_batch_size

0.167536
0.19134
0.182961
0.200914
0.166481
0.18901
0.18806
0.165333
0.183184
0.19052
0.188898
0.182952
0.161439
0.168668
0.180057
0.176598
0.199147
0.157843
0.198335
0.19307


In [58]:
correct = tf.equal(real_output, y)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
print(sess.run([accuracy],feed_dict={X: xd, y: yd}))

[0.0]


In [59]:
print(sess.run([real_output],feed_dict={X: xd, y: yd}))

[array([[ 0.611404  ],
       [ 0.61801881],
       [ 0.62460148],
       [ 0.63115221],
       [ 0.63734317],
       [ 0.64251858],
       [ 0.64650023],
       [ 0.65043199],
       [ 0.65432096],
       [ 0.65789437],
       [ 0.66114408],
       [ 0.66418988],
       [ 0.66718888],
       [ 0.67013872],
       [ 0.67304146],
       [ 0.67589653],
       [ 0.67870331],
       [ 0.68146312],
       [ 0.68417531],
       [ 0.68683958],
       [ 0.68945658],
       [ 0.69202626],
       [ 0.69454849],
       [ 0.69702345],
       [ 0.69945109],
       [ 0.70183206],
       [ 0.70416552],
       [ 0.70645285],
       [ 0.70869273],
       [ 0.710886  ],
       [ 0.71303231],
       [ 0.71513224],
       [ 0.71718574],
       [ 0.71919304],
       [ 0.72115362],
       [ 0.72306824],
       [ 0.72493672],
       [ 0.72675931],
       [ 0.72853577],
       [ 0.73026669],
       [ 0.73195207],
       [ 0.73359156],
       [ 0.7351861 ],
       [ 0.73673457],
       [ 0.73823857],
       [ 

In [56]:
yd

array([ 0.        ,  0.        ,  0.01171875,  0.0234375 ,  0.03515625,
        0.046875  ,  0.05859375,  0.05859375,  0.0703125 ,  0.08203125,
        0.09375   ,  0.10546875,  0.1171875 ,  0.12890625,  0.12890625,
        0.140625  ,  0.15234375,  0.1640625 ,  0.17578125,  0.1875    ,
        0.19921875,  0.19921875,  0.2109375 ,  0.22265625,  0.234375  ,
        0.24609375,  0.2578125 ,  0.26953125,  0.26953125,  0.28125   ,
        0.29296875,  0.3046875 ,  0.31640625,  0.328125  ,  0.33984375,
        0.33984375,  0.3515625 ,  0.36328125,  0.375     ,  0.38671875,
        0.3984375 ,  0.3984375 ,  0.41015625,  0.421875  ,  0.43359375,
        0.4453125 ,  0.45703125,  0.46875   ,  0.46875   ,  0.48046875,
        0.4921875 ,  0.50390625,  0.515625  ,  0.52734375,  0.5390625 ,
        0.5390625 ,  0.55078125,  0.5625    ,  0.57421875,  0.5859375 ,
        0.59765625,  0.609375  ,  0.609375  ,  0.62109375,  0.6328125 ,
        0.64453125,  0.65625   ,  0.66796875,  0.6796875 ,  0.67

In [81]:
BITS = np.array([0,0,0,0,0,0,0,1])
tf.reset_default_graph()
bits_test = tf.placeholder(tf.float32,shape=[None,8])
real_test = tf_bits2real(bits_test, bits=8,radius=1.5)
sess = tf.Session()
test_out = sess.run([real_test],feed_dict={bits_test:BITS.reshape([1,-1])})
print(test_out)
val_out = bits2real(BITS)
print(val_out)

[array([[ 0.01171875]], dtype=float32)]
0.01171875


In [23]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None,8],name='Input')
y = tf.placeholder(tf.float32, shape=[None,1],name='Correct_output')
output = tf_bits2real(X, bits=8,radius=1.5)
loss = output - y
mean_loss = tf.reduce_mean(loss)

In [24]:
# xd_ = np.arange(-1.49, 1.49, 0.01)
xd_ = np.arange(-1.49, 1.49, 0.01)
# np.random.shuffle(xd)
xd = np.zeros([xd_.shape[0],8])
yd= np.zeros([xd_.shape[0],1])
for i, x in enumerate(xd_):
    xd[i,:] = real2bits(x, bits=8, radius=1.5)
    yd[i] = bits2real(xd[i,:], bits=8, radius=1.5)

In [25]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [28]:
ml = sess.run([mean_loss],feed_dict={X:xd,y:yd})
print(ml)

[0.0]
